Цель:
Проанализировать вакансии на предмет их релевантности для специалиста "Фронтенд-разработчик".

Задача:
Для каждой профессии в датасете определить, подходит ли она для человека, ищущего работу по направлению "Фронтенд-разработка" (включая смежные и специализированные направления, связанные с созданием пользовательских интерфейсов, веб-разработкой, UI/UX).

Входные данные:
Файл с колонками:

title — название профессии или вакансии.

annotator_relevant — поле, которое необходимо заполнить аннотаторам.

relevant — скрытая "истинная" метка (используется для валидации).

Выходные метки:
1 (relevant) — профессия напрямую связана с фронтенд-разработкой: программирование интерфейсов, верстка, JavaScript/HTML/CSS, React, Vue, UI/UX, веб-приложения и т.п.

0 (not_relevant) — профессия не связана с фронтендом: менеджмент, DevOps, бекенд, аналитика, тестирование, системное администрирование, digital-маркетинг, прочие.

Инструкция:
Прочитайте внимательно значение в поле title. На его основе решите:

Ставьте метку 1, если:

Это фронтенд-разработчик или близкий синоним (например: Web Developer, UI Developer, React Developer).

Это специалист по HTML/CSS/JS/React, дизайнер, UX/UI с упором на веб-интерфейсы.

Это фуллстек, но с выраженной обязанностью по фронтенду (в названии есть “Frontend” или "Web").

Ставьте метку 0, если:

Профессия не связана с веб-разработкой (например, “Data Scientist”, “DevOps Engineer”, “Product Manager”, “SEO Specialist”).

В названии присутствует слово "Developer", но это не про веб или интерфейсы (например: “Android Developer”, “Embedded Engineer”, “Machine Learning Engineer”).

Смежная специальность (например: “Graphic Designer”, если нет акцента на UI; “Project Manager” даже в IT-команде).

В случае сомнений:

Оценивайте, будет ли специалист-фронтендер считать вакансию своей.

Лучше ошибиться в сторону "0", если вакансия не явно про фронтенд.

Валидация:
В датасет добавлены тестовые кейсы с заранее известной релевантностью (столбец relevant).

После аннотации метки будут сравниваться с истинными значениями.

Метрика качества — точность разметки на тестовых кейсах и/или согласованность между двумя аннотаторами (например, через Cohen's Kappa).

Примеры:

title	annotator_relevant	Пояснение
Frontend Developer	1	Прямо по профилю
Web Designer	1	Упор на веб-интерфейсы
Data Scientist	0	Не связана с фронтендом
Product Manager	0	Руководящая, не техническая
UI Developer	1	Входит в сферу фронтенда
Android Developer	0	Мобильная разработка


In [11]:
import pandas as pd

df = pd.read_csv("frontend_dataset.csv")

test_cases = [
    {"title": "(Test Case) Frontend Developer", "relevant": 1, "annotator_relevant": 1},
    {"title": "(Test Case) Backend Developer", "relevant": 0, "annotator_relevant": 0},
    {"title": "(Test Case) Web Developer", "relevant": 1, "annotator_relevant": 1},
    {"title": "(Test Case) Plumber", "relevant": 0, "annotator_relevant": 0},
    {"title": "(Test Case) JavaScript Developer", "relevant": 1, "annotator_relevant": 1}
]

test_df = pd.DataFrame(test_cases)
df = pd.concat([df, test_df], ignore_index=True)

df.to_csv("frontend_dataset_with_test_cases.csv", index=False)


✅ Файл сохранён как frontend_dataset_with_test_cases.csv


In [ ]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score

df = pd.read_csv("annotated_frontend_professions.csv")

# 1. Проверка на наличие пустых значений в ключевых столбцах
print("Пустые значения в столбцах:")
print(df.isnull().sum())

df_cleaned = df.dropna(subset=["title", "relevant", "annotator_relevant"])

# 2. Проверка на дубликаты
print(f"\nКоличество дубликатов в датасете: {df_cleaned.duplicated().sum()}")

df_cleaned = df_cleaned.drop_duplicates()

# 3. Анализ некорректных меток
incorrect_annotations = df_cleaned[df_cleaned["relevant"] != df_cleaned["annotator_relevant"]]
print(f"\nНекорректные аннотации: {len(incorrect_annotations)}")
print(incorrect_annotations)

# 4. Оценка согласованности с использованием коэффициента Каппы (для тестовых кейсов)
test_cases = df_cleaned[df_cleaned["title"].isin(["Frontend Developer", "Backend Developer", "Web Developer", "Plumber", "JavaScript Developer"])]
annotator_1 = test_cases["relevant"].tolist()
annotator_2 = test_cases["annotator_relevant"].tolist()

kappa_score = cohen_kappa_score(annotator_1, annotator_2)
print(f"\nКоэффициент Каппы: {kappa_score}")

# 5. Дополнительная очистка данных
df_cleaned = df_cleaned[~df_cleaned["title"].isin(incorrect_annotations["title"])]

df_cleaned.to_csv("cleaned_frontend_dataset.csv", index=False)

Пустые значения в столбцах:
title                 0
relevant              0
annotator_relevant    0
dtype: int64

Количество дубликатов в датасете: 0

Некорректные аннотации (когда разметка аннотатора отличается от правильной): 19
                               title  relevant  annotator_relevant
7                      UX Researcher         0                 1.0
9         UI Implementation Engineer         1                 0.0
31                 Angular Developer         1                 0.0
39                   Unity Developer         0                 1.0
40   Static Site Generator Developer         1                 0.0
116         Email Template Developer         1                 0.0
133             Full Stack Developer         0                 1.0
136               Character Designer         0                 1.0
186               Creative Developer         1                 0.0
199                   SPAs Developer         1                 0.0
219      Browser Extension Devel